# Testowanie z [pytest](https://docs.pytest.org/en/latest/) - część 2

In [ ]:
# Upewnijmy się, że pakiety pytest i ipytest są zainstalowane
# ipytest jest wymagany do uruchamiania pytest wewnątrz notatników Jupyter
import sys

!{sys.executable} -m pip install pytest
!{sys.executable} -m pip install ipytest

# Są one potrzebne do uruchamiania pytest wewnątrz notatników Jupyter
import ipytest

ipytest.autoconfig()

## [`@pytest.fixture`](https://docs.pytest.org/en/latest/fixture.html#pytest-fixtures-explicit-modular-scalable)
Rozważmy implementację klasy `Person`, którą chcemy przetestować.

In [ ]:
class Person:
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.age = age

    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"

    @property
    def as_dict(self):
        return {"name": self.full_name, "age": self.age}

    def increase_age(self, years):
        if years < 0:
            raise ValueError("Nie można odmładzać ludzi :(")
        self.age += years

Możesz łatwo tworzyć reużywalny kod testowy, używając fixture'ów pytest. Jeśli wprowadzisz swoje fixture'y w pliku [_conftest.py_](https://docs.pytest.org/en/latest/fixture.html#conftest-py-sharing-fixture-functions), będą one dostępne dla wszystkich twoich przypadków testowych. Ogólnie rzecz biorąc, lokalizacja _conftest.py_ znajduje się w głównym katalogu twojego folderu _tests_.

In [ ]:
import pytest


@pytest.fixture()
def default_person():
    person = Person(first_name="John", last_name="Doe", age=82)
    return person

Następnie możesz wykorzystać fixture `default_person` w rzeczywistych przypadkach testowych.

In [ ]:
%%ipytest


def test_full_name(default_person): # Uwaga: używamy fixture jako argumentu przypadku testowego
    result = default_person.full_name
    assert result == 'John Doe'
    
    
def test_as_dict(default_person):
    expected = {'name': 'John Doe', 'age': 82}
    result = default_person.as_dict
    assert result == expected
    
    
def test_increase_age(default_person):
    default_person.increase_age(1)
    assert default_person.age == 83
    
    default_person.increase_age(10)
    assert default_person.age == 93
    
    
def test_increase_age_with_negative_number(default_person):
    with pytest.raises(ValueError):
        default_person.increase_age(-1)

Używając fixture, mogliśmy użyć tej samej `default_person` dla wszystkich naszych czterech przypadków testowych!

W teście `test_increase_age_with_negative_number` użyliśmy [`pytest.raises`](https://docs.pytest.org/en/latest/assert.html#assertions-about-expected-exceptions), aby zweryfikować, czy zgłaszany jest wyjątek.

## [`@pytest.mark.parametrize`](https://docs.pytest.org/en/latest/parametrize.html#pytest-mark-parametrize-parametrizing-test-functions)
Czasami chcesz przetestować tę samą funkcjonalność z wieloma różnymi danymi wejściowymi. `pytest.mark.parametrize` to rozwiązanie do definiowania wielu różnych danych wejściowych z oczekiwanymi wynikami. Rozważmy następującą implementację funkcji `replace_names`.

In [ ]:
def replace_names(original_str, new_name):
    """Zastępuje imiona (słowa pisane wielką literą) w oryginalnym ciągu znaków nowym imieniem"""
    words = original_str.split()
    manipulated_words = [new_name if w.istitle() else w for w in words]
    return " ".join(manipulated_words)

Możemy przetestować funkcję `replace_names` z wieloma danymi wejściowymi, używając `pytest.mark.parametrize`.

In [ ]:
%%ipytest


@pytest.mark.parametrize("original,new_name,expected", [
        ('this is Lisa', 'John Doe', 'this is John Doe'),
        ('how about Frank and Amy', 'John', 'how about John and John'),
        ('no names here', 'John Doe', 'no names here'),
    ])
def test_replace_names(original, new_name, expected):
    result = replace_names(original, new_name)
    assert result == expected